### TTS-CBDB Kinship disambiguation.

### Methodology:
1. Restrict kinship type: "子". Kinship code:
```Python
kin_code_list_son=['180','182','183','184','185','186','191','193','194','195','196','198',
                   '199','202','205','206','207','211','212','213','214','220','221','222',
                   '226','229','231','234','235','307','326','339','436','560','568','572','575',]
```
2. Construct a list of pairs using CBDB data: ego person's CBDB ID + kinship person's name (we have CBDB ID), e.g.,
```Python
[[0, '田照鄰'],
 [1, '安邡'],
 [1, '安邠'],
 [1, '安郊'],
 [1, '安邦'],
 [3, '安扶'],
 [4, '查沖之'],
 [4, '查循之'],
 [6, '柴貽範'],
 [12, '晁子與'],
 [10097, '宋璲']]
```
3. Construct pairs using TTS data: ego person's CBDB ID (we have this) + kinship person's name (we don't have CBDB ID), e.g., 
```Python
[[56812, '富察昌齡'],
 [56812, '富察科占'],
 [56812, '富察查納'],
 [56813, '邵桓'],
 [56814, '刁錄'],
 [56814, '刁鈞'],
 [56814, '刁安仁'],
 [56814, '刁錦'],
 [56815, '邵鐸'],
 [56816, '于廷翼'],
 [10097, '宋璲']]
```
very similar to #2, but we don't have CBDB's ID for the kinship persons.
4. find the intersection of the two lists, in the example, it is `[10097, '宋璲']`, then we give `宋璲`'s CBDB ID to TTS, i.e., `120940`.
5. Kinship type "子" resolves 233 pairs of TTS records (total: 7983 pairs). 3%, not that bad.

In [1]:
% matplotlib inline
import sqlite3
import pandas as pd
import networkx as nx
import xlrd
import matplotlib.pyplot as plt
import math
import warnings
from tqdm import tqdm
warnings.filterwarnings('ignore')
plt.style.use('ggplot')

In [2]:
# Read data.
conn = sqlite3.connect('../data_file/20170424CBDBauUserSqlite.db')
df_kin_data=pd.read_sql_query("SELECT * FROM KIN_DATA", conn)
df_biog_main=pd.read_sql_query("SELECT * FROM BIOG_MAIN", conn).set_index('c_personid')
df_kinship_codes=pd.read_sql_query("SELECT * FROM KINSHIP_CODES", conn).set_index('c_kincode')
df_kin_tts=pd.read_excel('../data_file/110305 Kinship data (TTS_MQ for Deng Ke).xls', sheet_name='DATA')

In [3]:
# Get Top 10 kinship relationship in TTS dataset.
df_kin_tts.groupby('this is the relationship of the kin to EGO in Chinese').count()[['line_id']].sort_values(by='line_id', ascending=False)[0:10]

,line_id
this is the relationship of the kin to EGO in Chinese,
子,7983
孫,3925
父,3262
弟,1998
兄,1481
祖父,1443
曾孫,1276
曾祖父,931
祖,658


In [7]:
df_kin_tts.sample(3)

,line_id,CBDB PersonID of EGO,Name in Chinese of EGO; this is the main subject.,sys_id,This is the kin who is related to EGO.full name of kin,Kin Last Name,Kin FirstName,this is the relationship of the kin to EGO in Chinese,this is the kinship_code for the relationship of the kin to EGO
4916,5021,57696,富察達洪阿,926,富察福成,富察,福成,姪孫,240
7942,8148,58502,張誠基,1757,張錦峰,張,錦峰,子,180
999,1016,56938,楊錫紱,151,楊相輔,楊,相輔,曾祖父,48


In [9]:
df_kin_data.sample(3)

,tts_sysno,c_personid,c_kin_id,c_kin_code,c_source,c_pages,c_notes,c_autogen_notes,c_created_by,c_created_date,c_modified_by,c_modified_date
377205,375364.0,236935,132053,255,32046.0,第三甲第二名,None,None,load,20130923,None,None
329084,327243.0,206101,213321,375,32079.0,第三甲第二十一名,None,None,load,20130923,None,None
302223,300382.0,203246,301924,125,32069.0,第三甲第二百一十四名,None,None,load,20130923,None,None


In [10]:
df_kinship_codes.sample(3)

,c_kin_pair1,c_kin_pair2,c_kin_pair_notes,c_kinrel_chn,c_kinrel,c_kinrel_alt,c_pick_sorting,c_upstep,c_dwnstep,c_marstep,c_colstep
c_kincode,,,,,,,,,,,
4,304,304,None,同族長輩,K-n,"Lineage ancestor, generation undefined",50.0,2,1,0,2
100,181,-10000,None,妾之父,CF,Father of Consort,NaN,1,0,1,0
240,64,469,None,姪孫;從孫,BSS,BSS BG+2,9999.0,0,2,0,1


#### First let's try various types of sons.

In [84]:
##### Construct Father-Son pair from CBDB.
kin_code_list_son=[s for s in df_kinship_codes.index if '子' in str(df_kinship_codes.loc[s]['c_kinrel_chn'])]

df_kin_data_son=df_kin_data[df_kin_data['c_kin_code'].isin(kin_code_list_son)]

father_son_list_cbdb=[]
for index in tqdm(df_kin_data_son.index):
    try:
        father_c_personid=df_kin_data_son.loc[index, 'c_personid']
        son_name=df_biog_main.loc[df_kin_data_son.loc[index, 'c_kin_id'], 'c_name_chn']
        father_son_list_cbdb.append([father_c_personid, son_name])
    except:
        father_c_personid=df_kin_data_son.loc[index, 'c_personid']
        father_son_list_cbdb.append([father_c_personid, 'son_name_NULL'])

##### Construct Father-Son pair from TTS.
df_kin_son_tts=df_kin_tts.loc[[index for index in df_kin_tts.index if '子' in df_kin_tts.loc[index]['this is the relationship of the kin to EGO in Chinese']]]
father_son_list_tts=[]
for index in tqdm(df_kin_son_tts.index):
    try:
        father_c_personid=df_kin_son_tts.loc[index, 'CBDB PersonID of EGO']
        son_name=df_kin_son_tts.loc[index, 'This is the kin who is related to EGO.full name of kin']
        father_son_list_tts.append([father_c_personid, son_name])
    except:
        father_c_personid=df_kin_son_tts.loc[index, 'CBDB PersonID of EGO']
        father_son_list_tts.append([father_c_personid, 'son_name_NULL'])

##### Get the number of intersections, i.e., number of CBDB IDs to be assigned to TTS.
len(set([str(s) for s in father_son_list_cbdb]).intersection([str(s) for s in father_son_list_tts]))

100%|██████████| 9163/9163 [00:00<00:00, 23521.31it/s]


251

#### Let's build the process in bulk.

In [6]:
print('Kinship_recognized'+'\t'+'#Records_recognized')
for kr_str in df_kin_tts.groupby('this is the relationship of the kin to EGO in Chinese').count()[['line_id']].sort_values(by='line_id', ascending=False)[0:20].index:
    # KR = kinship to be recognized.
    print(kr_str+'\t', end='')
    ##### Construct KR pair from CBDB.
    kin_code_list_kr=[s for s in df_kinship_codes.index if kr_str in str(df_kinship_codes.loc[s]['c_kinrel_chn'])]
    df_kin_data_kr=df_kin_data[df_kin_data['c_kin_code'].isin(kin_code_list_kr)]
    kr_list_cbdb=[]
    for index in df_kin_data_kr.index:
        try:
            c_personid=df_kin_data_kr.loc[index, 'c_personid']
            kr_name=df_biog_main.loc[df_kin_data_kr.loc[index, 'c_kin_id'], 'c_name_chn']
            kr_list_cbdb.append([c_personid, kr_name])
        except:
            c_personid=df_kin_data_kr.loc[index, 'c_personid']
            kr_list_cbdb.append([c_personid, 'kr_name_NULL'])

    ##### Construct KR pair from TTS.
    df_kin_kr_tts=df_kin_tts.loc[[index for index in df_kin_tts.index if kr_str in df_kin_tts.loc[index]['this is the relationship of the kin to EGO in Chinese']]]
    kr_list_tts=[]
    for index in df_kin_kr_tts.index:
        try:
            c_personid=df_kin_kr_tts.loc[index, 'CBDB PersonID of EGO']
            kr_name=df_kin_kr_tts.loc[index, 'This is the kin who is related to EGO.full name of kin']
            kr_list_tts.append([c_personid, kr_name])
        except:
            c_personid=df_kin_kr_tts.loc[index, 'CBDB PersonID of EGO']
            kr_list_tts.append([c_personid, 'kr_name_NULL'])

    ##### Get the number of intersections, i.e., number of CBDB IDs to be assigned to TTS.
    print(len(set([str(s) for s in kr_list_cbdb]).intersection([str(s) for s in kr_list_tts])))

Kinship_recognized	#Records_recognized
子	251
孫	32
父	530
弟	111
兄	79
祖父	147
曾孫	2
曾祖父	0
祖	302
玄孫	1
曾祖	116
姪	3
祖先	0
高祖父	0
兄弟	1
後代	0
婿	15
母	4
叔	1
長子	1


### Draft.

In [ ]:
df_kinship_codes.to_excel('dump/df_kinship_codes.xlsx', encoding='utf8')

In [ ]:
len([s for s in df_tobe_done['this is the relationship of the kin to EGO in Chinese'] if '子' in s])